In [1]:
import numpy as np
import pandas as pd
import pysindy as ps
from sklearn.metrics import r2_score as sklearn_r2
import matplotlib.pyplot as plt
import math
# from pysindy.feature_library import FunctionLibrary

In [2]:
# Загрузка данных
data = pd.read_csv('generate_concentrations_1.csv')
t = np.arange(0, 60.5, 0.5)
C = data.values

print(len(C))
print(len(t))

121
121


In [6]:
# Загрузка данных
data = pd.read_csv('generate_concentrations_1.csv')
t = np.arange(0, 60.5, 0.5)
C = data.values

# Настройка PySINDy
poly_library = ps.PolynomialLibrary(degree=2)  # Полиномы до 2-й степени
custom_library = ps.CustomLibrary(library_functions=[np.tanh], function_names=["tanh"])
feature_library = ps.GeneralizedLibrary([poly_library, custom_library])
# feature_library = poly_library  # Без tanh
differentiation_method = ps.SmoothedFiniteDifference()
optimizer = ps.STLSQ(threshold=0.05)  # Порог для разреженности

model = ps.SINDy(
    differentiation_method=differentiation_method,
    feature_library=feature_library,
    optimizer=optimizer
)

# Обучение модели
model.fit(C, t=t)

# Вывод уравнений
print("Найденные уравнения:")
model.print()

# Оценка качества на производных
r2_deriv = model.score(C, t=t)
print(f"R^2 производных: {r2_deriv}")

# Симуляция траекторий
C_sim = model.simulate(C[0], t)

# Оценка R^2 для траекторий
r2_traj = sklearn_r2(C, C_sim, multioutput='raw_values')
print(f"R^2 траекторий по столбцам (C0, C1, C2, C3, C4): {r2_traj}")

# Визуализация
fig, axs = plt.subplots(5, 1, figsize=(10, 15), sharex=True)
labels = ['C0', 'C1', 'C2', 'C3', 'C4']
for i in range(5):
    axs[i].plot(t, C[:, i], 'b.', label=f'{labels[i]} данные')
    axs[i].plot(t, C_sim[:, i], 'r-', label=f'{labels[i]} аппроксимация')
    axs[i].set_ylabel(labels[i])
    axs[i].legend()
axs[-1].set_xlabel('Время (с)')
plt.tight_layout()
plt.show()

AttributeError: 'numpy.ufunc' object has no attribute '__code__'